Pablo Marchesi

January 2025

Mail: pablomarchesiselma@gmail.com

LinkedIn: https://www.linkedin.com/in/pablo-marchesi-010383199/

In [109]:
import numpy as np 
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

# Vasicek Model
def vasicek(params, T, dt, n_paths = 10**(4)):

    kappa = params['kappa']
    theta = params['theta']
    sigma = params['sigma']
    r0 = params['r0']

    n_steps = int(T/dt)
    Z = np.random.normal(size = (n_steps, n_paths))
    r_t = np.zeros((n_steps + 1, n_paths))
    r_t[0, :] = r0
    
    for t in range(1, n_steps + 1):
        dr = kappa * (theta - r_t[t-1, :]) * dt + sigma * np.sqrt(dt) * Z[t-1,:] 
        r_t[t, :] = r_t[t-1, :] + dr

    return r_t

# CIR Model
def CIR(params, T, dt, n_paths = 10**(4)):

    kappa = params['kappa']
    theta = params['theta']
    sigma = params['sigma']
    r0 = params['r0']

    n_steps = int(T/dt)
    Z = np.random.normal(size = (n_steps, n_paths))
    r_t = np.zeros((n_steps + 1, n_paths))
    r_t[0, :] = r0
    
    for t in range(1, n_steps + 1):
        dr = kappa * (theta - r_t[t-1, :]) * dt + sigma * np.sqrt(dt) * Z[t-1,:] * np.sqrt(r_t[t-1, :]) 
        r_t[t, :] = r_t[t-1, :] + dr

    return r_t

# Hull-White Model
def hull_white(params, T, dt, n_paths = 10**(4)):

    kappa = params['kappa']
    theta = params['theta']
    sigma = params['sigma']
    r0 = params['r0']

    n_steps = int(T/dt)
    Z = np.random.normal(size = (n_steps, n_paths))
    r_t = np.zeros((n_steps + 1, n_paths))
    r_t[0, :] = r0
    
    for t in range(1, n_steps + 1):
        dr = kappa * (theta(t-1, params) - r_t[t-1, :]) * dt + sigma * np.sqrt(dt) * Z[t-1,:] 
        r_t[t, :] = r_t[t-1, :] + dr

    return r_t

# Yield curve model 
def yield_curve(time, maturities, r_t, dt):

    yields = []
    time_index = int(time / dt)
    
    for T in maturities:
        T_index = int((time + T) / dt)
        if T_index >= r_t.shape[0]: break
        disc_fact = np.exp(-np.cumsum(r_t[time_index:T_index, :], axis=0) * dt)
        bond_price = np.mean(disc_fact[-1, :]) 
        long_rate = -np.log(bond_price) / T
        yields.append(long_rate)
    
    return np.array(yields)

# Empirical PDFs
def modelPDF(x, r_t):
    kde = gaussian_kde(r_t)
    pdf = kde(x)
    return pdf

In [248]:
# Model parameters
params = {'kappa': 0.1, 'theta': 0.05, 'sigma':0.01, 'r0': 0.03}
dt = 1 / 252; T = 20; n_paths = 10**(5); maturities = np.linspace(0.5, 10, 20) 

# Simulation of the vasicek model
short_rate = vasicek(params, T, dt, n_paths)

# Empirical PDF of the Vasicek model
x_vals = np.linspace(np.min(short_rate[-1,:]), np.max(short_rate[-1,:]), 5000)
pdf_vasicek = modelPDF(x_vals, short_rate[-1,:])

# Plots 
fig1 = px.line(x = x_vals, y = pdf_vasicek)

fig2 = go.Figure()
for t in range(0, 5):
    yields = yield_curve(t, maturities, short_rate, dt)
    fig2.add_trace(go.Scatter(x = maturities, y = yields, mode = 'lines', name = f"t = {t}"))

fig = make_subplots(rows=1, cols=2, subplot_titles=("PDF at Maturity", "Yield Curves"))
fig.add_traces(fig1.data, rows=1, cols=1); fig.update_xaxes(title_text="Short Rate", row=1, col=1)
fig.add_traces(fig2.data, rows=1, cols=2); fig.update_xaxes(title_text="Maturity", row=1, col=2)
fig.update_yaxes(title_text="Yield", row=1, col=2)

In [249]:
# Model parameters
params = {'kappa': 0.1, 'theta': 0.05, 'sigma':0.01, 'r0': 0.03}
dt = 1 / 252; T = 20; n_paths = 10**(5); maturities = np.linspace(0.5, 10, 20) 

# Simulation of the CIR model
short_rate = CIR(params, T, dt, n_paths)

# Empirical PDF of the CIR model
x_vals = np.linspace(np.min(short_rate[-1,:]), np.max(short_rate[-1,:]), 5000)
pdf_CIR = modelPDF(x_vals, short_rate[-1,:])

# Plots 
fig1 = px.line(x = x_vals, y = pdf_CIR)

fig2 = go.Figure()
for t in range(0, 5):
    yields = yield_curve(t, maturities, short_rate, dt)
    fig2.add_trace(go.Scatter(x = maturities, y = yields, mode = 'lines', name = f"t = {t}"))

fig = make_subplots(rows=1, cols=2, subplot_titles=("PDF at Maturity", "Yield Curves"))
fig.add_traces(fig1.data, rows=1, cols=1); fig.update_xaxes(title_text="Short Rate", row=1, col=1)
fig.add_traces(fig2.data, rows=1, cols=2); fig.update_xaxes(title_text="Maturity", row=1, col=2)
fig.update_yaxes(title_text="Yield", row=1, col=2)

In [145]:
from scipy.interpolate import interp1d

# Input maturities and yields
maturities = [1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30]  # Maturities in years
yields = [4.294, 4.278, 4.333, 4.296, 4.279, 4.159, 4.240, 4.270, 4.380, 4.479, 4.573, 4.862, 4.786]  # Yields in %

yields = [y / 100 for y in yields]

# Interpolate the spot yield curve
yield_curve_int = interp1d(maturities, yields, kind='cubic', bounds_error=False, fill_value=(yields[0], yields[-1]))

# Function to compute the derivative of the yield curve
def yield_curve_der(T, h=1e-5):
    return (yield_curve_int(T + h) - yield_curve_int(T - h)) / (2 * h)

# Function to compute the forward rate
def forward_rate(T):
    y = yield_curve_int(T)
    y_T_der = yield_curve_der(T, h=1e-5)
    return y + T * y_T_der

# Function to compute theta
def theta_fun(t, params):
    
    t_years = t / 365.0
    kappa = params['kappa']
    sigma = params['sigma']

    # Forward rate and its derivative
    f_M_t = forward_rate(t_years)
    h = 1e-5
    df_M_dt = (forward_rate(t_years + h) - forward_rate(t_years - h)) / (2 * h)
    
    # Theta calculation
    theta_t = df_M_dt + kappa * f_M_t + (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * t_years))
    return theta_t


In [151]:
# Model parameters
params = {'kappa': 0.05, 'theta': theta_fun, 'sigma':0.03, 'r0': 0.043}
dt = 1 / 252; T = 20; n_paths = 10**(5); maturities = np.linspace(0.5, 10, 20) 

# Simulation of the H-W model
short_rate = hull_white(params, T, dt, n_paths)

# Empirical PDF of the H-W model
x_vals = np.linspace(np.min(short_rate[-1,:]), np.max(short_rate[-1,:]), 5000)
pdf_HW = modelPDF(x_vals, short_rate[-1,:])

# Plots 
fig1 = px.line(x = x_vals, y = pdf_HW)

fig2 = go.Figure()
for t in range(0, 5):
    yields = yield_curve(t, maturities, short_rate, dt)
    fig2.add_trace(go.Scatter(x = maturities, y = yields, mode = 'lines', name = f"t = {t}"))

fig = make_subplots(rows=1, cols=2, subplot_titles=("PDF at Maturity", "Yield Curves"))
fig.add_traces(fig1.data, rows=1, cols=1); fig.update_xaxes(title_text="Short Rate", row=1, col=1)
fig.add_traces(fig2.data, rows=1, cols=2); fig.update_xaxes(title_text="Maturity", row=1, col=2)
fig.update_yaxes(title_text="Yield", row=1, col=2)
fig.show()


In [150]:
import pandas as pd

# Plot yield and forward curves
maturities = np.linspace(0.5, 29.999, 1000) 

df = pd.DataFrame({'Spot Curve': yield_curve_int(maturities), 'Forward Curve': forward_rate(maturities)}, index = maturities)
fig3 = px.line(df, title="Spot and Forward Curves", labels={"index": "Maturity (Years)", "value": "Yield"})
fig3.show()